# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from workflow_widgets import WorkflowWidget, WorkerPoolWidget, TailWidget
from droplet_workflow import droplet_wf
from example_workflow import example_wf
from lammps_workflow import lammps_wf
from aux_widgets import QueueWidget, NEWTAuthWidget, SSHAuthWidget, SSHTerminal, Space
import workflow_objects as kale

import ipywidgets as ipw
import time
from concurrent.futures import ThreadPoolExecutor
import traitlets
import networkx as nx
import IPython.display as disp

In [3]:
wpw = WorkerPoolWidget()
wpw

2017-10-24 12:19:24,734 INFO Performing db tune-up
2017-10-24 12:19:24,745 INFO LaunchPad was RESET.


A Jupyter Widget

In [5]:
droplet_wfw = WorkflowWidget(droplet_wf, wpw)
droplet_wfw

A Jupyter Widget

# Define simple workflow

In [6]:
wfw = WorkflowWidget(example_wf, wpw)
wfw

A Jupyter Widget

# Observe Output

In [7]:
t = TailWidget('out.txt')
t

A Jupyter Widget

# LAMMPS example

In [8]:
lammps_wfw = WorkflowWidget(lammps_wf, wpw)
lammps_wfw

A Jupyter Widget

# Connect to NEWT

In [9]:
a = NEWTAuthWidget()
a

A Jupyter Widget

In [10]:
q = QueueWidget(a)
q

A Jupyter Widget